# Orquestrador UFRB CETENS - Notebook Jupyter

Este notebook implementa a metodologia Scrappy para scraping de compartilhamentos do Grok, utilizando Playwright para extração dinâmica de conversas e organização em Markdown.

## 1. Instalação das Bibliotecas Necessárias

In [ ]:
# Instalar Playwright e dependências
!pip install playwright
!playwright install
# Para stealth: !pip install playwright-stealth (opcional)

## 2. Configuração do Playwright com Bypass de Proteções

In [ ]:
from playwright.async_api import async_playwright

async def setup_browser():
    playwright = await async_playwright().start()
    browser = await playwright.chromium.launch(
        headless=True,
        args=['--no-sandbox', '--disable-setuid-sandbox'],
        proxy={'server': 'https://us-pr.oxylabs.io:10000', 'username': 'customer-USERNAME', 'password': 'PASSWORD'}  # Substituir credenciais
    )
    page = await browser.new_page()
    await page.set_extra_http_headers({
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
    })
    return playwright, browser, page

## 3. Acesso à Página do Compartilhamento do Grok

In [ ]:
url = 'https://grok.com/share/c2hhcmQtMg%3D%3D_34aaa09f-7b10-486e-a5b3-3415cfb90f6b'
playwright, browser, page = await setup_browser()
await page.goto(url)
await page.wait_for_timeout(10000)  # Aguardar JS
print("Página carregada com sucesso")

## 4. Extração do Conteúdo Textual da Conversa

In [ ]:
content = await page.inner_text('body')
print("Conteúdo extraído:")
print(content[:500])  # Preview

## 5. Análise e Estruturação dos Dados Extraídos

In [ ]:
import re

# Identificar perguntas e respostas (simplificado)
questions = re.findall(r'Pergunta: (.+)', content)
responses = re.findall(r'Resposta: (.+)', content)
metadata = re.findall(r'Data: (.+)', content)

structured_data = {
    'questions': questions,
    'responses': responses,
    'metadata': metadata
}
print(structured_data)

## 6. Organização em Markdown Estruturado

In [ ]:
markdown_output = f"""
# Conversa Extraída do Grok

**Metadados:** {', '.join(structured_data['metadata'])}

## Perguntas e Respostas
"""

for i, (q, r) in enumerate(zip(structured_data['questions'], structured_data['responses'])):
    markdown_output += f"""
### {i+1}. Pergunta
{q}

### Resposta
{r}
"""

print(markdown_output)

## 7. Validação e Refinamento dos Dados

In [ ]:
# Validação simples
if len(structured_data['questions']) == len(structured_data['responses']):
    print("Dados consistentes. Saída final:")
    print(markdown_output)
else:
    print("Inconsistência detectada. Refinar extração.")

await browser.close()
await playwright.stop()